In [ ]:
# The code was removed by Watson Studio for sharing.

# Toronoto Neighborhoods



## About  

A real estate development company is interested to build in Toronto.  The stakeholders has brought us in to help with understanding the neighborhoods of the Canadian City, specifically the existing places of interest and local makeup.  

During this priliminary look at Toronto, we will be using Foursquare API to gather information on venues of various boroughs in the City. Other sources of data will be from web-scraping and files already downloaded.  


### Data Sources  
* Foursquare API
* Web-scraping
* Exisitng Files  


### Data Analysis  
* pandas
* folium 
* matplotlib

Part of IBM Data Science Specialization on Coursera

In [1]:
import sys
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install geopy
!{sys.executable} -m pip install folium
!{sys.executable} -m pip install beautifulsoup4
!{sys.executable} -m pip install lxml
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install sklearn
import numpy as np
import pandas as pd
import requests
import json
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
from bs4 import BeautifulSoup
from pandas.io.json import json_normalize

print('libraries imported')


libraries imported


### Web-scraping with Beautifulsoup  

We will start by getting the information of Toronto's postal codes and their corresponding neighborhood and borough names.  Postal codes are essential not only in identifying specific neighborhoods, but also in terms of mapping.  

  
We'll immediately turn the scraped data into a pandas dataframe. 

In [2]:
#raw data thru web scrape and transform to dataframe
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(url, 'lxml')

table = soup.find('table')
column_names = ['Postalcode','Borough','Neighborhood']
column_names
df = pd.DataFrame(columns = column_names)
# df.head()

for tr in table.find_all('tr'):
    rows=[]
    for td in tr.find_all('td'):
#         print(td)
        rows.append(td.text.strip())
    if len(rows)==3:
        df.loc[len(df)] = rows

df.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


#### Cleaning  

  
After we've gathered the web-scraped data into a pandas dataframe, we need to take a look at it to see if there are any unnecessary or missing information.  We'll remove accordingly.  We'll also do a bit of organizing as well. 

In [3]:
#clean data - check 'not assigned' and remove
index_names = df[ df['Borough'] =='Not assigned'].index

df.drop(index_names , inplace = True)
df.loc[df['Neighborhood'] =='Not assigned' , 'Neighborhood'] = df['Borough']
result = df.groupby(['Postalcode','Borough'], sort=False).agg( ', '.join)
df_new = result.reset_index()
df_new.head(10)

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


Let's take a look at the dimensions of this dataframe (rows, columns)

df_new.shape

### CSV to pandas    
We have the geospatial data (coordinates) for Toronto and its postal codes in our files already.  Let's convert the data into a pandas dataframe for analysis.  

In [4]:
# get coordinate data, make to dataframe
df_c = pd.read_csv('https://cocl.us/Geospatial_data')
df_c.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [5]:
df_c.columns = ['Postalcode', 'Latitude', 'Longitude']
# df_c.head()
# merge with web scraped data
df_comb = pd.merge(df_new, df_c, on='Postalcode')
df_comb.head(10)

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [6]:
df_new = df_comb
df_new.shape

(103, 5)

### Mapping with Folium  

Now that we have coordinate information on Toronot's neighborhoods/postal codes, we can go ahead and map it out with Folium, a map plotting Pytho plug-in, to see how things look.   

In [7]:
# info for mapping Toronto
address = 'Toronto'
geolocator = Nominatim(user_agent='tor_exp')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

In [31]:
# create map using latitude and longitude values
tor_map = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map of e borough
for lat, lng, label in zip(df_new['Latitude'], df_new['Longitude'], df_new['Borough']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(tor_map)  
    
tor_map

In [9]:
# The code was removed by Watson Studio for sharing.

In [10]:
thood = df_new[df_new['Borough'].str.contains('Toronto')].reset_index(drop=True)
# thood.shape
thood.head(10)

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259


In [11]:
neighborhood_name = thood.loc[0, 'Borough']
neighborhood_latitude = thood.loc[0, 'Latitude']
neighborhood_longitude = thood.loc[0, 'Longitude']

# check formatting
print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Downtown Toronto are 43.6542599, -79.3606359.


### Foursquare API  

Next, we'll go ahead and use the Foursquare API to get information on venues in each borough/postal code.   Since we have the coordinates of the boroughs, we'll use that data for our call.  

In [12]:
# Make foursq api call, set parameters
radius = 500
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)

In [13]:
# resulting data from api call
results = requests.get(url).json()
results;

#### Retreiving Relevant Data  

A lot of information is in the JSON data returned. We need to get the pieces that are relevant to our analysis, so let's go ahead and create a function to get the **venue category** of each result objects. 

In [14]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

After creating the function, we'll need to normalize the JSON data before turning it into a pandas dataframe. "

In [15]:
venues = results['response']['venues']
# venues
nearby_venues = json_normalize(venues)
nearby_venues

# # filter columns
filtered_columns = ['name', 'categories', 'location.lat', 'location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
# nearby_venues
# # # filter the category for each row
nearby_venues['categories'] = nearby_venues.apply(get_category_type, axis=1)

# # # clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head()

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,City Hall Council Chambers,City Hall,43.651827,-79.383949
1,Toronto City Hall,City Hall,43.653140,-79.383967
2,City of Toronto Civic Innovation Office,City Hall,43.653454,-79.383952
3,GoodLife Fitness Toronto Bell Trinity Centre,Gym,43.653436,-79.382314
4,Starbucks (Market at Longo's),Coffee Shop,43.653989,-79.384055


#### Repeating Calls  
We were able to get results for one postal code, now we need to repeat this process for all of the boroughs in Toronto.  For that, let's create a function based on what we did a few lines ago (in getting the nearby venues JSON).  <br> <br>
In addition to retrieving the JSON data, we can also process it into pandas dataframe. 

In [16]:
# function to get info on all venue data results
def getNearbyVenues(names, latitudes, longitudes, radius = 100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
#         print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [17]:
# make api call
# get venue info on boroughs (w toronto in name)
tor_venues = getNearbyVenues(names = thood['Borough'], latitudes = thood['Latitude'], longitudes = thood['Longitude'])
tor_venues.head()

,Borough,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Downtown Toronto,43.654260,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
1,Downtown Toronto,43.654260,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
2,Downtown Toronto,43.654260,-79.360636,Massimo's Kitchen Studio,43.654770,-79.359698,Italian Restaurant
3,Downtown Toronto,43.657162,-79.378937,Ryerson Image Centre,43.657523,-79.379460,Art Gallery
4,Downtown Toronto,43.657162,-79.378937,Balzac's Coffee,43.657854,-79.379200,Coffee Shop


Let's see the total amount of venues each borough has. 

In [18]:
# find total number venues for e/ 'toronto' neighborhood
tor_venues.groupby('Borough').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Borough,,,,,,
Central Toronto,15,15,15,15,15,15
Downtown Toronto,80,80,80,80,80,80
East Toronto,1,1,1,1,1,1
West Toronto,7,7,7,7,7,7


### Analyzing Categorical Data -  One Hot Encoding 

Since we are working with categorical data, we'll need to convert it into a numeric format so the computer can read it too!  To do that, we use **one hot encoding** where the categorical data is interpreted as binary (0, 1).

In [19]:
# one hot encoding - categorical data to computer-readable fmt
onehot = pd.get_dummies(tor_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Neighborhood'] = tor_venues['Borough'] 

# move neighborhood column to the first column
fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1])
onehot = onehot[fixed_columns]
onehot.head()

,Neighborhood,American Restaurant,Art Gallery,Asian Restaurant,Bakery,Bank,Bar,Beer Store,Bookstore,Breakfast Spot,...,Supermarket,Sushi Restaurant,Taco Place,Tea Room,Thai Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Yoga Studio
0,Downtown Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Downtown Toronto,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,Downtown Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Downtown Toronto,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Downtown Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


One hot encoding will allow us to see what kinds of venues (and their counts) are in each neighborhood/borough.  With this insight we can understand what venues are more common than others, and what we can interpret from this as well.  <br> <br>
Let's create a function to find out the most common venues in each borough.

In [20]:
# find out most common venue - create function
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

For the interest of this analysis, we will get the 10th most common venue in each borough and organize that in a pandas dataframe.

In [21]:
# normalize data
grouped = onehot.groupby('Neighborhood').mean().reset_index()
grouped
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe - common venue
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Borough'] = grouped['Neighborhood']

for ind in np.arange(grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(10)
# neighborhoods_venues_sorted.shape

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Café,Dessert Shop,Park,Liquor Store,Furniture / Home Store,Coffee Shop,Roof Deck,Italian Restaurant,Seafood Restaurant,Gym / Fitness Center
1,Downtown Toronto,Coffee Shop,Italian Restaurant,Restaurant,Café,Japanese Restaurant,Gym,Deli / Bodega,Pub,Salad Place,Sandwich Place
2,East Toronto,Trail,Yoga Studio,Cocktail Bar,Gastropub,Garden,Furniture / Home Store,Food Court,Fast Food Restaurant,Farmers Market,Dessert Shop
3,West Toronto,Yoga Studio,Japanese Restaurant,Asian Restaurant,Bar,Beer Store,Record Shop,Brewery,Concert Hall,Cupcake Shop,Deli / Bodega


### K-Means Clustering   

 <br>
The fun doesn't stop there!   <br> <br>
Let's dig further into these boroughs by using some machine learning.  We'll use K-Means Clustering to see the different (general) characteristics of the boroughs, how they are grouped in general.  <br> <br>
K-Means Clustering is typically used for categorical data to for segmentation, creating profiles of each cluster/segmentation based on similiar characteristics. 

In [22]:
# set number of clusters
kclusters = 4

grouped_clustering = grouped.drop('Neighborhood', 1)
# grouped_clustering.head()

# # # run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# # # check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 3, 1, 2], dtype=int32)

In [23]:
# insert cluster labels data to new col
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
neighborhoods_venues_sorted.head()

,Cluster Labels,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,Central Toronto,Café,Dessert Shop,Park,Liquor Store,Furniture / Home Store,Coffee Shop,Roof Deck,Italian Restaurant,Seafood Restaurant,Gym / Fitness Center
1,3,Downtown Toronto,Coffee Shop,Italian Restaurant,Restaurant,Café,Japanese Restaurant,Gym,Deli / Bodega,Pub,Salad Place,Sandwich Place
2,1,East Toronto,Trail,Yoga Studio,Cocktail Bar,Gastropub,Garden,Furniture / Home Store,Food Court,Fast Food Restaurant,Farmers Market,Dessert Shop
3,2,West Toronto,Yoga Studio,Japanese Restaurant,Asian Restaurant,Bar,Beer Store,Record Shop,Brewery,Concert Hall,Cupcake Shop,Deli / Bodega


#### Combing Dataframes and Visualizing Results  <br>
After applying K-Means Clustering model, we get a nice grouping of the boroughs.   <br> <br>

Since we have the coordinate information on the boroughs, we can combine it with this new dataframe (K-Means Clustering results based on most common venues) and visualize the results.  We can see how the clusters of mapped out in Toronto. 



In [24]:
# join cluster label dataframe w thood df
merged = thood
merged = merged.join(neighborhoods_venues_sorted.set_index('Borough'), on='Borough')
merged.head()

merged['Cluster Labels'] = merged['Cluster Labels'].fillna(0).astype(int)

In creating the map, we can distinguish each borough cluster with a different color - to do this, we'll create a "random color generator" and pass it through folium to map out the colors. 

In [30]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(merged['Latitude'], merged['Longitude'], merged['Neighborhood'], merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### K-Means Clustering Results  

In [26]:
merged.loc[merged['Cluster Labels'] == 0, merged.columns[[1] + list(range(5, merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,0,Café,Dessert Shop,Park,Liquor Store,Furniture / Home Store,Coffee Shop,Roof Deck,Italian Restaurant,Seafood Restaurant,Gym / Fitness Center
19,Central Toronto,0,Café,Dessert Shop,Park,Liquor Store,Furniture / Home Store,Coffee Shop,Roof Deck,Italian Restaurant,Seafood Restaurant,Gym / Fitness Center
20,Central Toronto,0,Café,Dessert Shop,Park,Liquor Store,Furniture / Home Store,Coffee Shop,Roof Deck,Italian Restaurant,Seafood Restaurant,Gym / Fitness Center
21,Central Toronto,0,Café,Dessert Shop,Park,Liquor Store,Furniture / Home Store,Coffee Shop,Roof Deck,Italian Restaurant,Seafood Restaurant,Gym / Fitness Center
23,Central Toronto,0,Café,Dessert Shop,Park,Liquor Store,Furniture / Home Store,Coffee Shop,Roof Deck,Italian Restaurant,Seafood Restaurant,Gym / Fitness Center
24,Central Toronto,0,Café,Dessert Shop,Park,Liquor Store,Furniture / Home Store,Coffee Shop,Roof Deck,Italian Restaurant,Seafood Restaurant,Gym / Fitness Center
26,Central Toronto,0,Café,Dessert Shop,Park,Liquor Store,Furniture / Home Store,Coffee Shop,Roof Deck,Italian Restaurant,Seafood Restaurant,Gym / Fitness Center
29,Central Toronto,0,Café,Dessert Shop,Park,Liquor Store,Furniture / Home Store,Coffee Shop,Roof Deck,Italian Restaurant,Seafood Restaurant,Gym / Fitness Center
31,Central Toronto,0,Café,Dessert Shop,Park,Liquor Store,Furniture / Home Store,Coffee Shop,Roof Deck,Italian Restaurant,Seafood Restaurant,Gym / Fitness Center


In [27]:
merged.loc[merged['Cluster Labels'] == 1, merged.columns[[1] + list(range(5, merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,East Toronto,1,Trail,Yoga Studio,Cocktail Bar,Gastropub,Garden,Furniture / Home Store,Food Court,Fast Food Restaurant,Farmers Market,Dessert Shop
12,East Toronto,1,Trail,Yoga Studio,Cocktail Bar,Gastropub,Garden,Furniture / Home Store,Food Court,Fast Food Restaurant,Farmers Market,Dessert Shop
15,East Toronto,1,Trail,Yoga Studio,Cocktail Bar,Gastropub,Garden,Furniture / Home Store,Food Court,Fast Food Restaurant,Farmers Market,Dessert Shop
17,East Toronto,1,Trail,Yoga Studio,Cocktail Bar,Gastropub,Garden,Furniture / Home Store,Food Court,Fast Food Restaurant,Farmers Market,Dessert Shop
38,East Toronto,1,Trail,Yoga Studio,Cocktail Bar,Gastropub,Garden,Furniture / Home Store,Food Court,Fast Food Restaurant,Farmers Market,Dessert Shop


In [28]:
merged.loc[merged['Cluster Labels'] == 2, merged.columns[[1] + list(range(5, merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,West Toronto,2,Yoga Studio,Japanese Restaurant,Asian Restaurant,Bar,Beer Store,Record Shop,Brewery,Concert Hall,Cupcake Shop,Deli / Bodega
11,West Toronto,2,Yoga Studio,Japanese Restaurant,Asian Restaurant,Bar,Beer Store,Record Shop,Brewery,Concert Hall,Cupcake Shop,Deli / Bodega
14,West Toronto,2,Yoga Studio,Japanese Restaurant,Asian Restaurant,Bar,Beer Store,Record Shop,Brewery,Concert Hall,Cupcake Shop,Deli / Bodega
22,West Toronto,2,Yoga Studio,Japanese Restaurant,Asian Restaurant,Bar,Beer Store,Record Shop,Brewery,Concert Hall,Cupcake Shop,Deli / Bodega
25,West Toronto,2,Yoga Studio,Japanese Restaurant,Asian Restaurant,Bar,Beer Store,Record Shop,Brewery,Concert Hall,Cupcake Shop,Deli / Bodega
28,West Toronto,2,Yoga Studio,Japanese Restaurant,Asian Restaurant,Bar,Beer Store,Record Shop,Brewery,Concert Hall,Cupcake Shop,Deli / Bodega


In [29]:
merged.loc[merged['Cluster Labels'] == 3, merged.columns[[1] + list(range(5, merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,3,Coffee Shop,Italian Restaurant,Restaurant,Café,Japanese Restaurant,Gym,Deli / Bodega,Pub,Salad Place,Sandwich Place
1,Downtown Toronto,3,Coffee Shop,Italian Restaurant,Restaurant,Café,Japanese Restaurant,Gym,Deli / Bodega,Pub,Salad Place,Sandwich Place
2,Downtown Toronto,3,Coffee Shop,Italian Restaurant,Restaurant,Café,Japanese Restaurant,Gym,Deli / Bodega,Pub,Salad Place,Sandwich Place
3,Downtown Toronto,3,Coffee Shop,Italian Restaurant,Restaurant,Café,Japanese Restaurant,Gym,Deli / Bodega,Pub,Salad Place,Sandwich Place
5,Downtown Toronto,3,Coffee Shop,Italian Restaurant,Restaurant,Café,Japanese Restaurant,Gym,Deli / Bodega,Pub,Salad Place,Sandwich Place
6,Downtown Toronto,3,Coffee Shop,Italian Restaurant,Restaurant,Café,Japanese Restaurant,Gym,Deli / Bodega,Pub,Salad Place,Sandwich Place
7,Downtown Toronto,3,Coffee Shop,Italian Restaurant,Restaurant,Café,Japanese Restaurant,Gym,Deli / Bodega,Pub,Salad Place,Sandwich Place
8,Downtown Toronto,3,Coffee Shop,Italian Restaurant,Restaurant,Café,Japanese Restaurant,Gym,Deli / Bodega,Pub,Salad Place,Sandwich Place
10,Downtown Toronto,3,Coffee Shop,Italian Restaurant,Restaurant,Café,Japanese Restaurant,Gym,Deli / Bodega,Pub,Salad Place,Sandwich Place
13,Downtown Toronto,3,Coffee Shop,Italian Restaurant,Restaurant,Café,Japanese Restaurant,Gym,Deli / Bodega,Pub,Salad Place,Sandwich Place
